# OpenStreetMap Data Wrangling Project

### Data extraction script

* ** Program: ** Data Analyst Nanodegree 
* ** Student: ** Guillermo Naranjo
* ** Date:** August 14, 2017

### Objective
Extract OSM XML data for selected geolocation, apply a basic data quality process to validate and transform nodes, ways and attribute tags to predefined entities in CSV files. 

### Brief
Based on OSM Case Study, the following script reads the OSM data file specified in OSM_PATH constant and extract the tags related to Nodes, Ways and its child tags. 
* Extracted data is organized in a single dictionary structure that is  **validated (1)** against the schema provided (schema.py) in order to comply minimum metadata and datatypes requirements.
* Each tag key is also **validated (2)** to discart problematic characters that would affect CSV formats using *is_problematic* method.
* A third basic data quality process is applied to each attribute to **validate (3)** the correct data type in order to set it correcty using *right_type* method.
* Finally, a text separator in included to prevent any text inconsistency once CSV is created using *insert_text_separator* method when required, **validate (4)**.
* I also verify if each required attribute key exists and has a valid value from extracted OSM file, for eample, each node tag always has the elements specified in NODE_FIELDS;  **if 'key' in element.attrib**  in *get_tags*

In [1]:
# ================================================== #
#     Importing required libraries                   #
# ================================================== #
import xml.etree.cElementTree as ET
from collections import defaultdict
import codecs
import cerberus
from unicode_dict_writer import unicode_dict_writer as udw
import schema
import re
import pprint

In [2]:
# ================================================== #
#     Global constants are defined                   #
# ================================================== #
OSM_PATH = "costa_rica_greater_metropolitan_area.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

SCHEMA = schema.schema

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

In [3]:
# ================================================== #
#               Validity Functions                   #
# ================================================== #

def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema """
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

def is_problematic(kvalue):
    """ Validates if key value contains problematic characters based on PROBLEMCHARS regex """
    return PROBLEMCHARS.search(kvalue)  

def insert_text_separator(text):
    """ Transforms text values to UTF-8 """
    return (text).encode('utf-8')

def right_type(key,value):
    """ Base on the item list, set the correct value datatype """
    if key in ['id','uid','changeset','version']:
        return int(value)
    elif key in ['lat','lon']:
        return float(value)
    else:
        return insert_text_separator(value)

In [4]:
# ================================================== #
#               Parser Functions                     #
# ================================================== #

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """ Parse xml tree and filter to only include node, ways and relation nodes """
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def get_key_type(k,default_tag_type='regular'):    
    """ Parse attribute k names to set the correct name and type if : exists in the name """
    klist = k.split(':')
    newkey = newtype = ""
    if len(klist) > 2:
        newkey = klist[1] + ":" + klist[2]
        newtype = klist[0]
    elif len(klist) == 2:
        newkey = klist[1]
        newtype = klist[0]
    else:
        newkey = klist[0]
        newtype = default_tag_type
    return newkey,newtype

def get_tags(_id, elements):
    """ get list of child tags dictionaries from node or ways tags """
    tags = []
    for tag in elements.iter("tag"):        
        if 'k' in tag.attrib:
            _key,_type = get_key_type(tag.attrib['k'])
            _value = tag.attrib['v'] 
            if not is_problematic(_value):                
                dtag = {}
                dtag['id'] = int(_id) 
                dtag['key'] = insert_text_separator(_key)
                dtag['value'] = insert_text_separator(_value)
                dtag['type'] = insert_text_separator(_type)
                tags.append(dtag)    
    return tags

def get_nodes(id, elements):
    """ get list of child nodes dictionaries from ways tags """
    nodes = []
    i = 0
    for node in elements.iter("nd"):                
        dnode = {}
        dnode['id'] = int(id) 
        dnode['node_id'] = int(node.attrib['ref'])
        dnode['position'] = int(i) 
        nodes.append(dnode)        
        i+=1
    return nodes

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS):
    """ creates main dictionary structure based on OSM xml extract """
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []

    if element.tag == 'node':        
        
        for item in node_attr_fields:
            node_attribs[item] = right_type(item,element.attrib[item])
            
        tags = get_tags(node_attribs['id'],element)                                    
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        
        for item in way_attr_fields:
            way_attribs[item] = right_type(item,element.attrib[item])
            
        way_nodes = get_nodes(way_attribs['id'],element)
        tags = get_tags(way_attribs['id'],element)         

        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

In [5]:
# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
        codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
        codecs.open(WAYS_PATH, 'w') as ways_file, \
        codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
        codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = udw(nodes_file, NODE_FIELDS)
        node_tags_writer = udw(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = udw(ways_file, WAY_FIELDS)
        way_nodes_writer = udw(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = udw(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [6]:
if __name__ == "__main__":
    process_map(OSM_PATH, validate=True)
    print " OSM XML extraction and CSV transformation successful."

 OSM XML extraction and CSV transformation successful.
